In [1]:
import suffix_tree
import pandas as pd
import os
import json
import pickle
from read_file import read_file
import node_op
import sys
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

In [2]:
find_encode = 1
brute = 1
drop_last = 1
seg_method = 1
ignore_len = 2
MINIMAL_REPEAT = 2
"""
Variable: find_encode
- 0: Use specified encode
- 1: Choose from candidate
--------------------------
Variable: brute (Loop all combination)
- 0: Don't loop
- 1: loop
--------------------------
Variable: drop_last (Last pattern handling)
- 0: handle it
- 1: Only do once MSA
--------------------------
Variable: seg_method (Segmentation method)
- 0: Unique MT
- 1: Split by top repeat
--------------------------
Variable: ignore_len (ignore maximal repeat <= len)
- 0: Off
- 1: On
--------------------------
Variable: MINIMAL_REPEAT (Minimal repeat count)
"""

"\nVariable: find_encode\n- 0: Use specified encode\n- 1: Choose from candidate\n--------------------------\nVariable: brute (Loop all combination)\n- 0: Don't loop\n- 1: loop\n--------------------------\nVariable: drop_last (Last pattern handling)\n- 0: handle it\n- 1: Only do once MSA\n--------------------------\nVariable: seg_method (Segmentation method)\n- 0: Unique MT\n- 1: Split by top repeat\n--------------------------\nVariable: ignore_len (ignore maximal repeat <= len)\n- 0: Off\n- 1: On\n--------------------------\nVariable: MINIMAL_REPEAT (Minimal repeat count)\n"

In [3]:
def binary(string, length):
    while len(string) != length:
        string = '0' + string
    return string

sys.setrecursionlimit(1000000)

# Input File

In [4]:
#current_path = os.path.join(os.path.expanduser("~"), "jupyter", "Fix_DCADE_Output")
#current_path = os.path.join(os.path.expanduser("~"), "Desktop", "DCADE", "TAM")
#current_path = os.path.join(os.path.expanduser("~"), "Desktop", "test", "1")
current_path = os.path.join(os.path.expanduser("~"), "Desktop", "test", "2")
#current_path = os.path.join(os.path.expanduser("~"), "Desktop", "DCADE", "RISE_", "R01")
input_file_path = os.path.join(current_path, "Output", "TXT", "TableA.txt")
print(input_file_path)

C:\Users\rick\Desktop\test\2\Output\TXT\TableA.txt


In [5]:
f = read_file(input_file_path)
content = f[0]
recb_start = f[1]
recb_end = f[2]
tag = f[3]
ids = f[4]
classes = f[5]
pathid = f[6]
parentid = f[7]
tecid = f[8]
cecid = f[9]
encoding = f[10]
col = f[11]
others = f[12]

In [6]:
tec_dict = {}
unique_mt = []
for node in range(len(col)):
    if col[node] == 'MT':
        if tecid[node] not in tec_dict.keys():
            tec_dict[tecid[node]] = [node]
        else:
            tec_dict[tecid[node]].append(node)
for key in tec_dict.keys():
    if len(tec_dict[key]) == 1:
        unique_mt += tec_dict[key]
print("Unique MT's index:\n", unique_mt)

Unique MT's index:
 [3, 10, 11, 12, 13, 14, 184, 211, 236, 1457, 1458, 1467, 1487]


# Translate node in to unicode for suffix tree

Change `encode_option` to use different attribute to encode leafnodes.
<br><br>
Leafnodes will have same encoding if they have same attribute.

In [7]:
best = {'option':'000000000', 'score': 0}
if find_encode == 1:
    check_dict = {}
    if brute == 1:
        candidate = []
        for i in range(1, 512):
            tmp = binary('{0:b}'.format(i), 9)
            if len(node_op.find_all_indexes(tmp, '1')) > 2:
                candidate.append(tmp)
    else:
        #candidate = ['000110100', '000101010', '000101001']
        with open('./good_encode.txt', 'rb') as f:
            candidate = pickle.load(f)
    total_progress = len(candidate)
    progress = 0
    progress_line = [0, 1, 5, 10, 20, 30, 40, 50, 60, 70, 80, 90, 95, 99, 100, 101]
    line_count = 0
    check_dict = {}
    for option in candidate:
        progress += 1
        while (progress/float(total_progress))*100 >= progress_line[line_count]:
            print("="*80, "\nProgress:", progress_line[line_count], "%\t", progress, "/", total_progress)
            print("="*80)
            line_count += 1
        encode_option = option
        encode_col = [tag, ids, classes, pathid, parentid, tecid, cecid, encoding, col]
        """
        Node op
        ==================================================
        """
        node_encode = node_op.encode_node(encode_col, encode_option, len(pathid))
        whole_string = node_encode[0]
        node_dict = node_encode[1] # code -> node num
        index_dict = node_encode[2] # code -> first index
        """
        ===================================================
        """
        
        inv_node_dict = {v: k for k, v in node_dict.items()} # node num -> code
        inv_index_dict = {v: k for k, v in index_dict.items()} # index num -> code
        if seg_method == 0:
            segments = node_op.segment_mt(unique_mt, whole_string)
            record_seg = node_op.mt_record_seg(segments, ignore_len, index_dict, inv_node_dict, MINIMAL_REPEAT)
        if seg_method == 1:
            segments, record_seg = node_op.segment_top(whole_string, ignore_len, index_dict, inv_node_dict, MINIMAL_REPEAT)
        
        all_seqs = node_op.get_all_seq(record_seg, segments)
        if len(record_seg) > 0 and str(all_seqs) not in check_dict.keys():
            check_dict[str(all_seqs)] = 1
            seq_score = []
            for seg_idx in range(len(all_seqs)):
                appear = {}
                score = 0.0
                length_min_max = [999, 0]
                for pattern in all_seqs[seg_idx][:-1]:
                    if len(pattern) < length_min_max[0]:
                        length_min_max[0] = len(pattern)
                    if len(pattern) > length_min_max[1]:
                        length_min_max[1] = len(pattern)
                
                score = np.amin(cosine_similarity(node_op.to_vector(all_seqs)[seg_idx]))
                if length_min_max[1] == 1 or length_min_max[0] == 1:
                    seq_score.append(score/10)
                else: seq_score.append(score)
            total_score = 0
            print(option)
            for s in range(len(record_seg)):
                print(s, record_seg[s][1], "\t%.2f" %(seq_score[s]))
                repeat_time = record_seg[s][1][0]
                total_score += seq_score[s]#/ pow(2, s)
            #print(all_seqs)
            print("Set count:", len(record_seg), "Score:", "%.2f" %(total_score/len(record_seg)))
            print('-'*80)
            average = total_score/len(record_seg)
            if average >= best['score']:
                print("\nBest Update\n")
                best['score'] = average
                best['Set count'] = len(record_seg)
                best['option'] = option
            """if repeat_time < best['repeat_time']:
                best['score'] = average
                best['Set count'] = len(record_seg)
                best['option'] = option
            elif repeat_time == best['repeat_time'] and total_score > best['score']:
                best['score'] = average
                best['Set count'] = len(record_seg)
                best['option'] = option"""
print("Best:", best)

Progress: 0 %	 1 / 466
000000111
0 (45, 'ĭŻį') 	0.17
1 (15, 'īĬĭĮį') 	0.43
Set count: 2 Score: 0.30
--------------------------------------------------------------------------------

Best Update

000001011
0 (160, 'bcb') 	0.55
1 (128, 'NNN') 	0.01
2 (6, 'jkl') 	0.43
3 (5, 'JJJ') 	0.08
4 (6, 'VUW') 	0.75
5 (4, 'AAA') 	0.09
6 (3, 'KKK') 	0.09
7 (4, '[\\[') 	0.83
Set count: 8 Score: 0.35
--------------------------------------------------------------------------------

Best Update

000001101
0 (45, 'ĲơĴ') 	0.20
1 (15, 'İıĲĳĴ') 	0.46
Set count: 2 Score: 0.33
--------------------------------------------------------------------------------
Progress: 1 %	 5 / 466
000010011
0 (989, 'CCC') 	0.00
1 (4, 'AAA') 	0.08
Set count: 2 Score: 0.04
--------------------------------------------------------------------------------
000011010
0 (160, 'aba') 	0.55
1 (128, 'MMM') 	0.01
2 (6, 'ijk') 	0.43
3 (5, 'AAA') 	0.08
4 (6, 'UTV') 	0.75
5 (5, 'III') 	0.08
6 (4, 'Z[Z') 	0.83
7 (3, 'JJJ') 	0.09
Set count: 8 Sc

In [8]:
if find_encode == 1:
    encode_option = best['option']
else:
    #encode_option = '000110100'
    encode_option = '000011010'

encode_col = [tag, ids, classes, pathid, parentid, tecid, cecid, encoding, col]
"""
Node op
==================================================
"""
node_encode = node_op.encode_node(encode_col, encode_option, len(pathid))
whole_string = node_encode[0]
node_dict = node_encode[1] # code -> node num
index_dict = node_encode[2] # code -> first index
"""
===================================================
"""
print("Example: ")

for col_num in range(len(encode_col)):
    if encode_option[col_num] == '1':
        print(encode_col[col_num][node], end='')
inv_node_dict = {v: k for k, v in node_dict.items()} # node num -> code
inv_index_dict = {v: k for k, v in index_dict.items()} # index num -> code
#print("\nConvert to Unicode String:\n", whole_string)

Example: 
1229-16MT

# Segment whole string

In [9]:
if seg_method == 0:
    segments = node_op.segment_mt(unique_mt, whole_string)
    record_seg = node_op.mt_record_seg(segments, ignore_len, index_dict, inv_node_dict, MINIMAL_REPEAT)
if seg_method == 1:
    segments, record_seg = node_op.segment_top(whole_string, ignore_len, index_dict, inv_node_dict, MINIMAL_REPEAT)

In [10]:
all_seqs = node_op.get_all_seq(record_seg, segments)
print(all_seqs)

[['cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdccef^_`ab', 'cd', 'cdcdef^_`abcaga', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef`abcagacdef^_`ab', 'cd', 'cdcdef`abcagacdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef^_`ab', 'cd', 'cdcdef`ab', 'cd', 'cdcdef^

# MSA

In [11]:
import cstar
removed_whole_string = whole_string
for seg_idx in range(len(all_seqs)):
    json_result = []
    print("First round MSA\n", "="*100)
    scores = [5, -4, -2] # matchScore, mismatchScore, gapScore
    if len(all_seqs[seg_idx][:-1]) == 1:
        msa = all_seqs[seg_idx][:-1]
    else:
        msa = cstar.CenterStar(scores, all_seqs[seg_idx][:-1]).msa()
    trans_dict = {}
    last_c = '-'
    end_idx = 0

    for i in range(len(msa)):
        if msa[i][-1] != '-' and last_c not in trans_dict.keys() and drop_last == 0:
            last_c = msa[i][-1]
            trans_dict[last_c] = msa[i].replace('-', '')
        if msa[i].replace('-', '') not in trans_dict.keys():
            trans_dict[msa[i].replace('-', '')] = msa[i]
        else: pass
    for i in all_seqs[seg_idx][:-1]: print(i, "\n\t\t-> ", trans_dict[i])
    if drop_last == 0:
        print('='*100, "\nSecond round MSA\n")
        msa_2 = cstar.CenterStar(scores, msa+[all_seqs[seg_idx][-1]]).msa()
        trans_dict_2 = {}

        for i in range(len(msa_2)):
            if msa_2[i].replace('-', '') not in trans_dict_2.keys():
                trans_dict_2[msa_2[i].replace('-', '')] = msa_2[i]
            else: pass
        for idx in range(len(trans_dict_2[trans_dict[last_c]])):
            if trans_dict_2[trans_dict[last_c]][idx] == last_c:
                end_idx = idx
        for i in all_seqs[seg_idx]:
            trans_dict_2[i] = trans_dict_2[i][:end_idx+1]
            tmp = trans_dict_2[i][:end_idx+1].replace('-', '')
            removed_whole_string = removed_whole_string.replace(tmp, '-'*len(tmp))
            print(i, "\n\t\t-> ", trans_dict_2[i])

        json_schema = [{} for i in range(len(trans_dict_2[list(trans_dict_2.keys())[0]]))]
        schema_check = [0 for i in range(len(trans_dict_2[list(trans_dict_2.keys())[0]]))]
    else:
        json_schema = [{} for i in range(len(trans_dict[list(trans_dict.keys())[0]]))]
        schema_check = [0 for i in range(len(trans_dict[list(trans_dict.keys())[0]]))]

    with open(os.path.join(current_path, "Output", "Set-" + str(seg_idx) + ".txt"), 'w', encoding='utf-8') as file:
        for page in range(len(others)):
            json_page = []
            output_dict = {} # Record which pattern is used
            if drop_last == 1:
                length = len(all_seqs[seg_idx]) - 1
            else:
                length = len(all_seqs[seg_idx])
            for s in range(length):
                write_tmp = []
                json_set = []
                write_tmp.append(str(page) + "-" + str(seg_idx) + "-" + str(s) + "\t")
                tmp = node_op.find_all_indexes(whole_string, all_seqs[seg_idx][s])
                if all_seqs[seg_idx][s] not in output_dict.keys():
                    output_dict[all_seqs[seg_idx][s]] = 0
                else:
                    output_dict[all_seqs[seg_idx][s]] += 1
                #print("start:", tmp[output_dict[seqs[s]]], others[page][tmp[output_dict[seqs[s]]]])
                idx = 0
                if drop_last == 0:
                    for c in range(len(trans_dict_2[all_seqs[seg_idx][s]])):
                        if trans_dict_2[all_seqs[seg_idx][s]][c] == '-':
                            write_tmp.append('\t')
                            json_set.append('')
                        else:
                            write_tmp.append(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx][:others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx].find(" ::")] + "\t")
                            json_set.append(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx][:others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx].find(" ::")])
                            if schema_check[c] == 0:
                                schema_check[c] = 1
                                json_schema[c]["PathId"] = pathid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["ParentId"] = parentid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx].split(':')[0].replace('\"', '')
                                if encoding[tmp[output_dict[all_seqs[seg_idx][s]]]+idx] == ' ':
                                    json_schema[c]["Encoding"] = ''
                                else:
                                    json_schema[c]["Encoding"] = int(encoding[tmp[output_dict[all_seqs[seg_idx][s]]]+idx])
                                json_schema[c]["CECId"] = cecid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["TECId"] = tecid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["ColType"] = col[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                            idx += 1
                    if len(list(c for c in write_tmp if c != '\t' and c != '')) != 1:
                        for word in write_tmp:
                            file.write(word)
                        file.write('\n')
                else:
                    for c in range(len(trans_dict[all_seqs[seg_idx][s]])):
                        if trans_dict[all_seqs[seg_idx][s]][c] == '-':
                            write_tmp.append('\t')
                            json_set.append('')
                        else:
                            write_tmp.append(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx][:others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx].find(" ::")] + "\t")
                            json_set.append(others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx][:others[page][tmp[output_dict[all_seqs[seg_idx][s]]]+idx].find(" ::")])
                            if schema_check[c] == 0:
                                schema_check[c] = 1
                                json_schema[c]["PathId"] = pathid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["ParentId"] = parentid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx].split(':')[0].replace('\"', '')
                                if encoding[tmp[output_dict[all_seqs[seg_idx][s]]]+idx] == ' ':
                                    json_schema[c]["Encoding"] = ''
                                else:
                                    json_schema[c]["Encoding"] = int(encoding[tmp[output_dict[all_seqs[seg_idx][s]]]+idx])
                                json_schema[c]["CECId"] = cecid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["TECId"] = tecid[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                                json_schema[c]["ColType"] = col[tmp[output_dict[all_seqs[seg_idx][s]]]+idx]
                            idx += 1
                    if len(list(c for c in write_tmp if c != '\t' and c != '')) != 1:
                        for word in write_tmp:
                            file.write(word)
                        file.write('\n')
                json_page.append(json_set)
            json_result.append(json_page)
    with open(os.path.join(current_path, "Output", "Set-" + str(seg_idx) + "_schema.json"), 'w') as json_file:
        json.dump(json_result, json_file)
    with open(os.path.join(current_path, "Output", "Set-" + str(seg_idx) + "_schema.json"), 'w') as json_file:
        json.dump(json_schema, json_file)

First round MSA
cd 
		->  -------------c---d-----------
cdcdef^_`ab 
		->  --cd---------c---def^_`ab----
cd 
		->  -------------c---d-----------
cdcdef^_`ab 
		->  --cd---------c---def^_`ab----
cd 
		->  -------------c---d-----------
cdcdef^_`ab 
		->  --cd---------c---def^_`ab----
cd 
		->  -------------c---d-----------
cdcdef^_`ab 
		->  --cd---------c---def^_`ab----
cd 
		->  -------------c---d-----------
cdccef^_`ab 
		->  --cd---------c---cef^_`ab----
cd 
		->  -------------c---d-----------
cdcdef^_`abcaga 
		->  --cd---------c---def^_`abcaga
cdcdef^_`ab 
		->  --cd---------c---def^_`ab----
cd 
		->  -------------c---d-----------
cdcdef^_`ab 
		->  --cd---------c---def^_`ab----
cd 
		->  -------------c---d-----------
cdcdef^_`ab 
		->  --cd---------c---def^_`ab----
cd 
		->  -------------c---d-----------
cdcdef^_`ab 
		->  --cd---------c---def^_`ab----
cd 
		->  -------------c---d-----------
cdcdef^_`ab 
		->  --cd---------c---def^_`ab----
cd 
		->  -------------c---d-----------
c

J 
		->  J
J 
		->  J
J 
		->  J
J 
		->  J
First round MSA
WVXY 
		->  WVXY
WVXY 
		->  WVXY
WVXZ 
		->  WVXZ
WVXY 
		->  WVXY
WVXY 
		->  WVXY
First round MSA
A 
		->  A
A 
		->  A
A 
		->  A
First round MSA
\] 
		->  \]
\] 
		->  \]
\] 
		->  \]


# Modified TableA Output

In [12]:
json_table = []
json_schema = []
for page in range(len(others)):
    json_page = []
    set_count = 0
    check = False
    for node in range(len(removed_whole_string)):
        schema_dict = {}
        if removed_whole_string[node] == '-':
            if check == False:
                check = True
                set_count += 1
                json_page.append(str(set_count) + '-' + str(page))
                schema_dict["PathId"] = ""
                schema_dict["ParentId"] = ""
                schema_dict["Encoding"] = -1
                schema_dict["CECId"] = ""
                schema_dict["TECId"] = ""
                schema_dict["ColType"] = "MR"
                json_schema.append(schema_dict)
            else: pass
        else:
            if check == True:
                check = False
            json_page.append(others[page][node][:others[page][node].find(" ::")])
            schema_dict["PathId"] = pathid[node]
            schema_dict["ParentId"] = parentid[node].split(':')[0].replace('\"', '')
            if encoding[node] != ' ':
                schema_dict["Encoding"] = int(encoding[node])
            else:
                schema_dict["Encoding"] = ''
            schema_dict["CECId"] = cecid[node]
            schema_dict["TECId"] = tecid[node]
            schema_dict["ColType"] = col[node]
            json_schema.append(schema_dict)
    json_table.append(json_page)
with open(os.path.join(current_path, "Output", "TableA.json"), 'w') as json_file:
    json.dump(json_table, json_file)
with open(os.path.join(current_path, "Output", "SchemaTableA.json"), 'w') as json_file:
    json.dump(json_schema, json_file)

# ===============================

In [13]:
with open('./good_encode.txt', 'rb') as f:
    candidate = pickle.load(f)
if best['option'] not in candidate and brute == 1:
    with open('./good_encode.txt', 'wb') as f:
        candidate.append(best['option'])
        print("Append:", best['option'])
        pickle.dump(candidate, f)

In [14]:
candidate

['000011010',
 '000001011',
 '000100011',
 '000001101',
 '000101001',
 '000110010',
 '000101010',
 '011000010',
 '000110001',
 '001010010',
 '000000111',
 '000010011',
 '000001110',
 '001000011']

In [15]:
if brute == 1:
    with open('./good_encode.txt', 'rb') as f:
        candidate = pickle.load(f)

In [16]:
#candidate = candidate[:-1]

In [17]:
if brute == 1:
    with open('./good_encode.txt', 'wb') as f:
        pickle.dump(candidate, f)

In [18]:
print(len(all_seqs), "Set\nBEST: ", best)

7 Set
BEST:  {'option': '000101001', 'score': 0.39385429209338063, 'Set count': 7}


In [19]:
ignore_len

2